In [41]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

start_time = time.time()

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)
    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()
B_DF = DTBS_DataFrame['B'].copy()
C_DF = DTBS_DataFrame['C'].copy()
D_DF = DTBS_DataFrame['D'].copy()

In [31]:
# 设置周一（或其他时间）换仓
start_day = '2023-07-01'
end_day = '2023-08-18'

set_which_day = 'mon'
name = '热门可转债+ytm（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
('热门可转债+ytm（全量数据DTBS，用于回测）.xlsx', 'mon')
"""


df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)
trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
print(trade_date_list)

output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
output_df['可转债名称'] = list(temp['cn'])
output_df['正股代码'] = list(temp['sc'])
output_df['正股名称'] = list(temp['sn'])
output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['个债收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']

output_df['组合收益率'] = output_df.groupby('买入日期')['卖出价'].transform('sum') / output_df.groupby('买入日期')['买入价'].transform('sum') - 1

output_df['买入日期时的中证转债指数'] = list(pd.merge(output_df,C_DF[['date','zi']],left_on = '买入日期',right_on='date',how='left')['zi'])
output_df['卖出日期时的中证转债指数'] = list(pd.merge(output_df,C_DF[['date','zi']],left_on = '卖出日期',right_on='date',how='left')['zi'])
output_df['同期中证转债收益率'] = output_df['卖出日期时的中证转债指数'] / output_df['买入日期时的中证转债指数'] - 1
output_df['组合收益率是否跑赢同期指数'] = output_df.apply(lambda row: '是' if row['组合收益率'] > row['同期中证转债收益率'] else '否', axis=1)


# 使用transform方法计算每个分组内的总和
grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')

# 计算每个分组的百分比
output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum


# 根据需要自行调整需要输出的列
output_df = output_df[['持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价','组合收益率','同期中证转债收益率', 
                       '组合收益率是否跑赢同期指数', '个债收益率', '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()

# output_df = output_df[['持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价','个债收益率', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()

print(output_df)

strategy_name = str(name[ : name.index('（')])
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}
output_excel_name = strategy_name + '（换仓日：' + mapping[set_which_day] + '，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

              日期      可转债代码  价格（收盘价全价）  正股名称       涨跌幅        双低    转股溢价率  \
0     2020-01-17  123032.SZ    106.204   万里马 -0.000922  123.2314  17.0274   
1     2020-01-17  123033.SZ    113.000  金力永磁 -0.002648  132.4050  19.4050   
2     2020-01-17  128042.SZ    110.099  凯中精密  0.006629  133.5807  23.4817   
3     2020-01-20  123032.SZ    105.880   万里马 -0.003051  124.8232  18.9432   
4     2020-01-20  123033.SZ    112.610  金力永磁 -0.003451  131.4504  18.8404   
...          ...        ...        ...   ...       ...       ...      ...   
4527  2023-08-18  113625.SH    117.959  江山欧派 -0.011083  185.7190  67.7600   
4528  2023-08-18  123082.SZ    113.600  北陆药业 -0.008657  166.4800  52.8800   
4529  2023-08-18  123151.SZ    114.360  康泰医学 -0.002964  159.9000  45.5400   
4530  2023-08-18  123171.SZ    117.162  共同药业 -0.007102  158.3620  41.2000   
4531  2023-08-18  123186.SZ    116.281  志特新材 -0.008290  192.6110  76.3300   

      债券余额（亿）   剩余期限（年）       换手率   到期收益率  qs  qs30  
0      1.8029  0.8715

In [42]:
# 查看所有换仓日的情况
start_day = '2023-05-01'
end_day = '2023-08-18'

set_which_days = ['mon', 'tue', 'wed', 'thu', 'fri']

name = '热门可转债+ytm（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
"""
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}

df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)

all_output_df = []

for set_which_day in set_which_days:
    
    trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
    print(trade_date_list)

    output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
    output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
    temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    output_df['可转债名称'] = list(temp['cn'])
    output_df['正股代码'] = list(temp['sc'])
    output_df['正股名称'] = list(temp['sn'])
    output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['个债收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
    output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']
    
    output_df['组合收益率'] = output_df.groupby('买入日期')['卖出价'].transform('sum') / output_df.groupby('买入日期')['买入价'].transform('sum') - 1
    
    # 使用transform方法计算每个分组内的总和
    grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')
    
    # 计算每个分组的百分比
    output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum
    
    output_df['换仓日'] = mapping[set_which_day]
    
    output_df = output_df[['换仓日', '持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '组合收益率','个债收益率', 
                           '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    
    
    
    ouput_last_df = df[df['日期']==trade_date_list[-1]][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    ouput_last_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    temp = pd.merge(ouput_last_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    ouput_last_df['可转债名称'] = list(temp['cn'])
    ouput_last_df['正股代码'] = list(temp['sc'])
    ouput_last_df['正股名称'] = list(temp['sn'])
    ouput_last_df['买入价'] = list(pd.merge(ouput_last_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    ouput_last_df['换仓日'] = mapping[set_which_day]
    
    output_df = pd.concat([output_df, ouput_last_df], ignore_index=True)
    
    all_output_df.append(output_df)
    print(output_df)

output_df = pd.concat(all_output_df, ignore_index=True)

output_df['买入日期时的中证转债指数'] = list(pd.merge(output_df,C_DF[['date','zi']],left_on = '买入日期',right_on='date',how='left')['zi'])
output_df['卖出日期时的中证转债指数'] = list(pd.merge(output_df,C_DF[['date','zi']],left_on = '卖出日期',right_on='date',how='left')['zi'])
output_df['同期中证转债收益率'] = output_df['卖出日期时的中证转债指数'] / output_df['买入日期时的中证转债指数'] - 1
output_df['组合收益率是否跑赢同期指数'] = output_df.apply(lambda row: '是' if row['组合收益率'] > row['同期中证转债收益率'] else '否', axis=1)

# 若同时期中正转债指数收益率为空值，
output_df.loc[output_df['同期中证转债收益率'].isnull(), '组合收益率是否跑赢同期指数'] = None

output_df.sort_values(by=['买入日期','可转债代码'],ascending=[True,True] ,inplace=True)

# 只保留需要输出的列
"""
output_df = output_df[['换仓日', '持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价','组合收益率','同期中证转债收益率', 
                       '组合收益率是否跑赢同期指数', '个债收益率', '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
"""

output_df = output_df[['换仓日', '持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '个债收益率', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()

strategy_name = str(name[ : name.index('（')])

output_excel_name = strategy_name + '（换仓日：周一至周五，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

             日期      可转债代码  价格（收盘价全价）  正股名称       涨跌幅        双低    转股溢价率  \
0    2020-01-17  128042.SZ    110.099  凯中精密  0.006629  133.5807  23.4817   
1    2020-01-20  128042.SZ    111.000  凯中精密  0.008184  136.0311  25.0311   
2    2020-01-22  123033.SZ    110.000  金力永磁 -0.012390  131.1765  21.1765   
3    2020-01-22  128069.SZ    112.599  华森制药 -0.023807  123.8444  11.2454   
4    2020-01-23  128069.SZ    111.230  华森制药 -0.012158  124.9765  13.7465   
..          ...        ...        ...   ...       ...       ...      ...   
611  2023-08-09  123082.SZ    115.664  北陆药业 -0.004159  156.6740  41.0100   
612  2023-08-10  123171.SZ    116.646  共同药业  0.000412  155.9260  39.2800   
613  2023-08-11  123082.SZ    115.700  北陆药业  0.001732  161.6700  45.9700   
614  2023-08-16  123082.SZ    115.229  北陆药业 -0.002631  162.7490  47.5200   
615  2023-08-17  123082.SZ    114.592  北陆药业 -0.005528  162.8320  48.2400   

      债券余额（亿）   剩余期限（年）       换手率   到期收益率  qs  qs30  
0    4.158700  4.532800  2.027500

程序运行时间：16.09秒


        A   B    C      D
0  Group1  10  100  220.0
1  Group1  20  200  220.0
2  Group2  15  150  275.0
3  Group2  25  250  275.0
4  Group1  30  300  220.0
5  Group2  35  350  275.0
